In [7]:
from flask import Flask, render_template
from markupsafe import escape

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text
from sqlalchemy.sql import func
import os

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(app.root_path, 'movie.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 
app.app_context().push()

db = SQLAlchemy(app)

# many to many relation
movie_actor_association = db.Table(
    'movie_actor_association',
    db.Column('movie_id', db.Integer, db.ForeignKey('movie_info.movie_id')),
    db.Column('actor_id', db.Integer, db.ForeignKey('actor_info.actor_id')),
)

movie_director_association = db.Table(
    'movie_director_association',
    db.Column('movie_id', db.Integer, db.ForeignKey('movie_info.movie_id')),
    db.Column('actor_id', db.Integer, db.ForeignKey('actor_info.actor_id')),
)

class Movie(db.Model):
    __tablename__ = 'movie_info'

    movie_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    movie_name = db.Column(db.String(20))
    release_date = db.Column(db.String(15))
    country = db.Column(db.String(20))
    movie_type = db.Column(db.String(10))
    release_year = db.Column(db.Integer)
    
    moviebox = db.relationship('MovieBox', back_populates='movies', uselist=False, cascade='all, delete-orphan')
    actors = db.relationship('Actor', secondary=movie_actor_association, back_populates='movie_actors')
    directors = db.relationship('Actor', secondary=movie_director_association, back_populates='movie_directs')
    
    def __repr__(self):
        return f'<Movie {self.movie_name}>'
    
class Actor(db.Model):
    __tablename__ = 'actor_info'

    actor_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    actor_name = db.Column(db.String(20))
    gender = db.Column(db.String(2))
    country = db.Column(db.String(20))
    
    movie_actors = db.relationship('Movie', secondary=movie_actor_association)
    movie_directs = db.relationship('Movie', secondary=movie_director_association)
    
    def __repr__(self):
        return f'<Actor {self.actor_name}>'    
    
# one to one relation
class MovieBox(db.Model):
    __tablename__ = 'movie_box'
    box_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    movie_id = db.Column(db.String(10), db.ForeignKey('movie_info.movie_id'))
    movie_box = db.Column(db.Float)
    movies = db.relationship('Movie', back_populates='moviebox')

movies = Movie.query.all()
movies

[<Movie 战狼2>,
 <Movie 哪吒之魔童降世>,
 <Movie 流浪地球>,
 <Movie 复仇者联盟4>,
 <Movie 红海行动>,
 <Movie 唐人街探案2>,
 <Movie 我不是药神>,
 <Movie 中国机长>,
 <Movie 速度与激情8>,
 <Movie 西虹市首富>,
 <Movie 复仇者联盟3>,
 <Movie 捉妖记2>,
 <Movie 八佰>,
 <Movie 姜子牙>,
 <Movie Example Movie>,
 <Movie Example Movie>,
 <Movie Example Movie>]

### 删除最后一部电影

In [8]:
movie_id = Movie.query.all()[-1].movie_id
def delete_movie(movie_id):
    # Step 1: Retrieve the movie by its ID
    movie = Movie.query.get_or_404(movie_id)

    if movie:
        if len(movie.actors)>0:
            for actor in movie.actors:
                db.session.delete(actor)
        if len(movie.directors)>0:
            for director in movie.directors:
                db.session.delete(director)
        if movie.moviebox:
            db.session.delete(movie.moviebox)
        db.session.delete(movie)
        try:
            db.session.commit()
            print(f"Movie with ID {movie_id} has been deleted.")
        except:
            db.session.rollback()
            print(f"Movie with ID {movie_id} not found.")
        
delete_movie(movie_id)
Movie.query.all()

Movie with ID 1022 has been deleted.


[<Movie 战狼2>,
 <Movie 哪吒之魔童降世>,
 <Movie 流浪地球>,
 <Movie 复仇者联盟4>,
 <Movie 红海行动>,
 <Movie 唐人街探案2>,
 <Movie 我不是药神>,
 <Movie 中国机长>,
 <Movie 速度与激情8>,
 <Movie 西虹市首富>,
 <Movie 复仇者联盟3>,
 <Movie 捉妖记2>,
 <Movie 八佰>,
 <Movie 姜子牙>,
 <Movie Example Movie>,
 <Movie Example Movie>]

### Add a Movie

In [6]:
def add_movie(movie_data):
    # Step 1: Create a new Movie object
    new_movie = Movie(
        movie_name=movie_data['movie_name'],
        release_date=movie_data['release_date'],
        country=movie_data['country'],
        movie_type=movie_data['movie_type'],
        release_year=movie_data['release_year']
    )

    # Step 2: Add any associated records or relationships
    # Add actors to the movie
    actors_data = movie_data.get('actors', [])
    for actor_data in actors_data:
        actor = Actor(**actor_data)
        new_movie.actors.append(actor)

    # Add directors to the movie
    directors_data = movie_data.get('directors', [])
    for director_data in directors_data:
        director = Actor(**director_data)
        new_movie.directors.append(director)

    # Add movie box details
    movie_box_data = movie_data.get('movie_box', {})
    if movie_box_data:
        movie_box = MovieBox(**movie_box_data)
        new_movie.moviebox = movie_box

    # Step 3: Add the movie to the database
    db.session.add(new_movie)
    db.session.commit()

    print(f"Movie '{new_movie.movie_name}' has been added with ID {new_movie.movie_id}.")

# Example usage:
movie_data = {
    'movie_name': 'Example Movie',
    'release_date': '2023-01-01',
    'country': 'Example Country',
    'movie_type': 'Action',
    'release_year': 2023,
    'actors': [
        {'actor_name': 'Actor 1', 'gender': 'M', 'country': 'Country 1'},
        {'actor_name': 'Actor 2', 'gender': 'F', 'country': 'Country 2'}
    ],
    'directors': [
        {'actor_name': 'Director 1', 'gender': 'M', 'country': 'Country 3'}
    ],
    'movie_box': {'movie_box': 1000000.00}
}

add_movie(movie_data)
Movie.query.all()

Movie 'Example Movie' has been added with ID 1022.


[<Movie 战狼2>,
 <Movie 哪吒之魔童降世>,
 <Movie 流浪地球>,
 <Movie 复仇者联盟4>,
 <Movie 红海行动>,
 <Movie 唐人街探案2>,
 <Movie 我不是药神>,
 <Movie 中国机长>,
 <Movie 速度与激情8>,
 <Movie 西虹市首富>,
 <Movie 复仇者联盟3>,
 <Movie 捉妖记2>,
 <Movie 八佰>,
 <Movie 姜子牙>,
 <Movie Example Movie>,
 <Movie Example Movie>,
 <Movie Example Movie>]

In [4]:
Movie.query.all()[-1].actors

[<Actor Actor 2>, <Actor Actor 1>]

In [67]:
db.session.rollback()